In [1]:
import numpy as np
import os
import time
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras import layers
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
#import tf.kerasTasm
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
# Loading the training data
from PIL import ImageOps
import shutil
from pathlib import Path
import pandas as pd

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# Define data path
data_path = r'/floyd/input/images/Number'
data_dir_list = os.listdir(data_path)
data_dir_list

df = pd.read_csv('/floyd/input/csv/images.csv', header=None)
arr = df[2].unique() 
df[2] = df[2].transform(lambda x: np.array(np.where(arr == x)).item())

desired_size = 224

img_data_list=[]
price_list = []
sales = np.array([])

for dataset in data_dir_list:
    img_list=os.listdir(os.path.join(data_path,dataset))
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))  
    for img in img_list:
        stem = Path(img).stem     
        img_path = os.path.join(data_path, dataset, img)
        img = image.load_img(img_path, target_size=(desired_size, desired_size))
        # img = img.resize((desired_size, desired_size))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)   
        
        price = np.array([
                df.loc[df[0] == stem, 5].iloc[0], #price
                df.loc[df[0] == stem, 2].iloc[0], #color family
                df.loc[df[0] == stem, 3].iloc[0], #class
                df.loc[df[0] == stem, 4].iloc[0], #style
                int(stem.split('_')[1]) #color
            ])
        
        price_list.append(price)
        img_data_list.append(x)
        img.close()       
    sales = np.append(sales, np.full((len(img_list),), int(dataset),dtype='float64') )

img_data = np.array(img_data_list)
price_data = np.array(price_list)


img_data=np.rollaxis(img_data,1,0)
img_data=img_data[0]

Loaded the images of dataset-1

Loaded the images of dataset-0



In [3]:
img_data, price_data, sales = shuffle(img_data, price_data, sales, random_state=2)
img_data_train, img_data_test, price_data_train, price_data_test, sales_train, sales_test = train_test_split(img_data, price_data, sales, test_size=0.1, random_state=2) # TAKES TIME
img_data_val, img_data_test, price_data_val, price_data_test, sales_val, sales_test = train_test_split(img_data_test, price_data_test, sales_test, test_size=0.5, random_state=2) # TAKES TIME

In [4]:
dataset = '1'
img_data_list=[]
price_list = []
img_list=os.listdir(os.path.join(data_path, dataset))
print(os.path.join(data_path,'1'))
print ('Loaded the images of dataset-1 for augmentation')  
for img in img_list:
    stem = Path(img).stem     
    img_path = os.path.join(data_path, dataset, img)
    img = image.load_img(img_path, target_size=(desired_size, desired_size))
    x = image.img_to_array(ImageOps.mirror(img))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)   
        
    price = np.array([
                df.loc[df[0] == stem, 5].iloc[0], #price
                df.loc[df[0] == stem, 2].iloc[0], #color family
                df.loc[df[0] == stem, 3].iloc[0], #class
                df.loc[df[0] == stem, 4].iloc[0], #style
                int(stem.split('_')[1]) #color
            ])
        
    price_list.append(price)
    img_data_list.append(x)
    img.close()
sales = np.full((len(img_list),), 1, dtype='float64')
img_data = np.array(img_data_list)
price_data = np.array(price_list)

img_data=np.rollaxis(img_data,1,0)
img_data=img_data[0]

/floyd/input/images/Number/1
Loaded the images of dataset-1 for augmentation


In [5]:
sales_train.reshape((sales_train.shape[0], 1))
sales.reshape((sales.shape[0], 1))
print(img_data_train.shape)
print(img_data.shape)
print(price_data_train.shape)
print(price_data.shape)
print(sales_train.shape)
print(sales.shape)

(9369, 224, 224, 3)
(3749, 224, 224, 3)
(9369, 5)
(3749, 5)
(9369,)
(3749,)


In [6]:
img_data_train = np.vstack((img_data_train, img_data))
price_data_train = np.vstack((price_data_train, price_data))
print(img_data_train.shape)
print(price_data_train.shape)

(13118, 224, 224, 3)
(13118, 5)


In [7]:
sales_train = np.append(sales_train, sales,axis = 0)
print(sales_train.shape)

(13118,)


In [8]:
img_data_train, price_data_train, sales_train = shuffle(img_data_train, price_data_train, sales_train, random_state=2)

In [9]:
image_input = Input(shape=(desired_size, desired_size, 3))

conv = InceptionV3(input_tensor=image_input, include_top=True, weights='imagenet')
image_encoding = conv.get_layer('predictions').output



price_input = Input(shape=(price_data_train.shape[1],))
price_encoding = Dense(10, activation='relu')(price_input)
price_encoding = Dense(10, activation='relu')(price_encoding)

concatenated = layers.concatenate([image_encoding, price_encoding], axis=-1)

sales_output = Dense(1, activation='sigmoid', name='output')(concatenated)
98
model = Model([image_input, price_input], sales_output)
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
96116736/96112376 [==============================] - 5s 0us/step


In [10]:
t=time.time()
hist = model.fit([img_data_train, price_data_train], sales_train, batch_size=32, epochs=40, verbose=1, validation_data=([img_data_val, price_data_val], sales_val))

print('Training time: %s' % (time.time() - t))

model.save('model.h5')

Train on 13118 samples, validate on 520 samples
Epoch 1/40
13118/13118 [==============================] - 83s 6ms/step - loss: 2.9700 - binary_accuracy: 0.5068 - val_loss: 2.2736 - val_binary_accuracy: 0.6462
Epoch 2/40
13118/13118 [==============================] - 75s 6ms/step - loss: 2.4699 - binary_accuracy: 0.4985 - val_loss: 0.6856 - val_binary_accuracy: 0.5865
Epoch 3/40
13118/13118 [==============================] - 76s 6ms/step - loss: 2.2512 - binary_accuracy: 0.4976 - val_loss: 4.3922 - val_binary_accuracy: 0.3500
Epoch 4/40
13118/13118 [==============================] - 76s 6ms/step - loss: 2.0904 - binary_accuracy: 0.4965 - val_loss: 0.8387 - val_binary_accuracy: 0.3673
Epoch 5/40
13118/13118 [==============================] - 76s 6ms/step - loss: 1.9793 - binary_accuracy: 0.5052 - val_loss: 2.6247 - val_binary_accuracy: 0.3481
Epoch 6/40
13118/13118 [==============================] - 75s 6ms/step - loss: 1.8465 - binary_accuracy: 0.5047 - val_loss: 1.1175 - val_binary_acc

NotImplementedError: numpy() is only available when eager execution is enabled.

In [11]:
from sklearn.metrics import classification_report,confusion_matrix
import itertools
#################################################################
sales_pred = model.predict([img_data_test, price_data_test]) # NO OF IMAGES x NUMBER OF CLASSES
sales_pred = np.round(sales_pred) # required to bring to proper format 1xcol


#%%
print(classification_report(sales_test, sales_pred)) #

              precision    recall  f1-score   support

         0.0       0.88      0.02      0.04       324
         1.0       0.38      0.99      0.55       197

    accuracy                           0.39       521
   macro avg       0.63      0.51      0.30       521
weighted avg       0.69      0.39      0.23       521



In [ ]:
#This model with 5 csv params gave around 68% on set of 30 and on 102-135 set 